<a href="https://colab.research.google.com/github/shamemezahid/EULA-Analyzer/blob/main/EULA_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting up google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Converts all the privacy policy page content into individual
 txt files.**

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

# creates a list of eula urls
file = open('/content/drive/MyDrive/EULA Project/Work/eula-links.csv', 'r')
eulas = file.read().split(',')

for eula in eulas:
  url = eula

  # skips webpage in case of an http error
  try:
    html = urlopen(url).read()
  except:
    continue
  
  # parses the html when success
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
    script.extract()    # rip it out
  
  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines()) 
  
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  # remove non english terms from the docs
  text = bytes(text, 'utf-8').decode('utf-8', 'ignore')

  # file path name generate
  path = '/content/drive/MyDrive/EULA Project/Work/Text-Eula/'+str(eulas.index(eula))+'.txt'
  
  # Write line to file
  with open(path, 'w') as writefile:
    writefile.write(text)

In [ ]:
wordlist_path = '/content/drive/MyDrive/EULA Project/Work/wordlist.csv'
file = open(wordlist_path, 'r')
wordlist = file.read().split(", ")

eulapath = '/content/drive/MyDrive/EULA Project/Work/Text-Eula/0.txt'
file = open(eulapath, 'r')
sentences = file.read().split('.')
flaggedsentences = []
for sentence in sentences[:]:
  added = False
  for word in wordlist:
    if word in sentence:
      added = True 
      flaggedsentences.append(sentence)
      continue
    elif(added): 
      break
flaggedsentences = '. '.join(flaggedsentences)
flaggedfilepath = '/content/drive/MyDrive/EULA Project/Work/Text-Eula/0f.txt'
file = open(flaggedfilepath, 'w')
file.write(flaggedsentences)

flaggedsentences

'Facebook\nFacebookEmail or phonePasswordForgot account?Sign UpPlain Text Terms of Service Facebook Ads Controls\nPrivacy Basics\nCookies Policy\nData Policy\nMore Resources\nView a printable version of the Terms of Service An update to our Terms of Service will be effective on July 26, 2022.  You can preview the updated version here.  We’ve updated our Terms of Use, Data Policy, and Cookies Policy to reflect the new name on January 4, 2022.  While our company name has changed, we are continuing to offer the same products, including the Facebook app from Meta.  Our Data Policy and Terms of Service remain in effect, and this name change does not affect how we use or share data. Terms of Service Meta builds technologies and services that enable people to connect with each other, build communities, and grow businesses.  These Terms govern your use of Facebook, Messenger, and the other products, features, apps, services, technologies, and software we offer (the Meta Products or Products), 

**Combining all the text**

In [ ]:
# combines all the collected eula into one 

# files are named as 0.txt to 50.txt
filename = 0
maxfile = 50
filepath = '/content/drive/MyDrive/EULA Project/Work/Text-Eula/'
filetype = '.txt'
# where the combined file will be
finalpath = '/content/drive/MyDrive/EULA Project/Work/Text-Eula/combined.txt'

# empties the destination file in case there it already existed because
# appending to an existing file will create duplicates in case of multiple run 
finalfile = open(finalpath, 'w')
finalfile.write("")
finalfile.close()

while filename < maxfile:
  # opens destination file in append mode 
  finalfile = open(finalpath, 'a')
  try:
    # checks if the current file exists  
    file = open(filepath+str(filename)+filetype, 'r')
  except:
    # if not, then next file name
    filename = filename + 1
    continue
  # content of the current file
  filecontent = file.read()
  # write the content to the destination file 
  finalfile.write(filecontent)
  # closing the files
  file.close()
  finalfile.close()
  # loop control
  filename = filename + 1

**Extraction Methods**

In [ ]:
# KEY-BERT Installation
!pip3 install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 235 kB 18.1 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
     |████████████████████████████████| 4.4 MB 58.0 MB/s 
     |████████████████████████████████| 1.2 MB 60.2 MB/s 
     |████████████████████████████████| 101 kB 14.3 MB/s 
     |████████████████████████████████| 596 kB 62.5 MB/s 
     |████████████████████████████████| 6.6 MB 32.5 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=06f473822e7c0f56001cc065ebf18374680999b650d25adf0868fc46be78ab17
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7fa04b0085fa781e3148b2d0466299576e24ac0b3ae41aa331a5e71ed81a414f
 

In [ ]:
# KEY-BERT
from keybert import KeyBERT

file = open('/content/drive/MyDrive/EULA Project/Work/Text-Eula/combined.txt', 'r')
doc = file.read()

keyword_model = KeyBERT()
keywords = keyword_model.extract_keywords(doc, keyphrase_ngram_range=(1,2), stop_words='english', top_n=100)
# print(keywords)
for keyword in keywords:
  print(keyword)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

('terms facebook', 0.6087)
('facebookemail phonepasswordforgot', 0.5948)
('service facebook', 0.5713)
('services facebook', 0.5655)
('account facebook', 0.5568)
('agreement facebook', 0.5431)
('facebook account', 0.5396)
('accounts facebook', 0.5338)
('facebook device', 0.5309)
('policythe facebook', 0.525)
('facebook use', 0.519)
('cookies facebook', 0.5181)
('facebook access', 0.5082)
('platform facebook', 0.5073)
('facebook meta', 0.5061)
('facebook regarding', 0.5047)
('facebook cookies', 0.5039)
('facebook products', 0.5016)
('ads facebook', 0.4991)
('facebook protect', 0.499)
('does facebook', 0.4979)
('phonepasswordforgot account', 0.4937)
('facebook login', 0.4933)
('example facebook', 0.4914)
('platforms facebook', 0.4909)
('facebook security', 0.4885)
('account apps', 0.4885)
('access facebook', 0.4876)
('request facebook', 0.4869)
('privacy facebook', 0.4865)
('messenger facebook', 0.4828)
('site facebook', 0.4812)
('facebook app', 0.4809)
('facebook marketplace', 0.4804)
('

In [ ]:
# Yake Installation
!pip3 install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.5 MB/s 
     |████████████████████████████████| 132 kB 16.7 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73987 sha256=45c37feb672ca02e1541a0ae8e1e89b7b94ff1996391d4b1266008d5460a9dbe
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
# Yet Another Keyword Extractor
import yake

file = open('/content/drive/MyDrive/EULA Project/Work/Text-Eula/combined.txt', 'r')
doc = file.read()

keyword_extractor = yake.KeywordExtractor()
keywords = keyword_extractor.extract_keywords(doc)
for kw in keywords:
  print(kw)

('User Personal Information', 8.733839010784628e-07)
('personal information', 1.255505279644316e-06)
('Privacy Policy', 1.5368738936888725e-06)
('personal data', 2.37732026835245e-06)
('personal Microsoft account', 3.796480581107677e-06)
('services', 5.379322388493448e-06)
('Google Privacy Policy', 5.686190538425635e-06)
('information', 6.058622872554669e-06)
('CCPA personal information', 6.638534450009505e-06)
('Game Services', 8.106132337044197e-06)
('Privacy Policy Privacy', 8.566456354151825e-06)
('Services Privacy Policy', 8.639922827337104e-06)
('Policy Privacy Policy', 1.2188246279326586e-05)
('Service', 1.2457073074215967e-05)
('collect Personal Information', 1.2719521286236278e-05)
('Data', 1.2876353738136549e-05)
('Epic Services', 1.5165308345840415e-05)
('Facebook Privacy Policy', 1.6356270396286864e-05)
('Privacy', 1.6375743248184273e-05)
('service providers', 1.665797553745724e-05)


In [ ]:
# stemming, lemmatizing, cleaning the texts 
!pip3 install nltk

from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

def stemming(raw_text):
  pst, lst = PorterStemmer(), LancasterStemmer()
  porter_stemmed_tokens = [pst.stem(tok) for tok in  raw_text]
  lancaster_stemmed_tokens = [lst.stem(tok) for tok in  raw_text]
  return porter_stemmed_tokens, lancaster_stemmed_tokens

def lematization(raw_text):
  lemmatizer = WordNetLemmatizer() 
  lematized_tokens = [lemmatizer.lemmatize(tok) for tok in  raw_text]
  return lematized_tokens

def clean_text_data(raw_text):
  # print(len(raw_text))
  text_lower = raw_text.lower()
  text_punc_removed = text_lower.translate(str.maketrans('', '', string.punctuation))
  # text_punc_removed = text_lower
  token = word_tokenize(text_punc_removed)

  stopwords = set(nltk.corpus.stopwords.words('english'))
  token_without_stopwords = [x for x in token if x not in stopwords]

  stemmed_text = lematization(token_without_stopwords)
  privacy_policies_clean_text = ' '.join(stemmed_text)
  return privacy_policies_clean_text

file = open('/content/drive/MyDrive/EULA Project/Work/Text-Eula/2.txt', 'r')
doc = file.read()

doc = lematization(doc)
print(doc)

# privacy_policies_text_clean = policies.raw_text.apply(lambda privacy_policy_text_raw: clean_text_data(privacy_policy_text_raw))
# privacy_policies_text_clean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


LookupError: ignored